In [ ]:
# Install required packages
# pip install langchain huggingface_hub pandas faiss-cpu numpy transformers torch accelerate bitsandbytes gradio

import pandas as pd
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import gradio as gr

def load_embeddings(csv_path):
    try:
        df = pd.read_csv(csv_path)
        
        if 'Cleaned_Ideas' not in df.columns or 'Embeddings' not in df.columns:
            raise ValueError("CSV must contain 'Cleaned_Ideas' and 'Embeddings' columns")
            
        df['Embeddings'] = df['Embeddings'].apply(
            lambda x: np.fromstring(
                x.strip("[]").replace("\n", ""),
                sep=", ",
                dtype=np.float32
            )
        )
        
        expected_dim = 768
        valid_embeddings = df['Embeddings'].apply(lambda x: len(x) == expected_dim)
        if not valid_embeddings.all():
            invalid_count = len(df) - valid_embeddings.sum()
            raise ValueError(f"{invalid_count} entries have invalid embedding dimensions")
            
        return df['Cleaned_Ideas'].tolist(), np.array(df['Embeddings'].tolist())
        
    except Exception as e:
        print(f"Error loading embeddings: {str(e)}")
        raise

# 1. Load texts and embeddings from CSV
texts, embeddings = load_embeddings("ideas_with_embeddings.csv")

# 2. Improved embedding model and FAISS configuration
try:
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vector_store = FAISS.from_embeddings(
        text_embeddings=list(zip(texts, embeddings)),
        embedding=embedding_model,
        normalize_L2=True
    )
    print(f"FAISS index created with {vector_store.index.ntotal} entries")
    
except Exception as e:
    print(f"Vector store creation failed: {str(e)}")
    raise

# 3. Using an openly available model (GPT-Neo 1.3B)
model_name = "EleutherAI/gpt-neo-1.3B"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    print("GPT-Neo 1.3B model loaded successfully.")
    
except Exception as e:
    print(f"Model loading failed: {str(e)}")
    print("You may need to accept the model license at: https://huggingface.co/EleutherAI/gpt-neo-1.3B")
    raise

# Text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    do_sample=True,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

# 4. Enhanced prompt template
template = """<context>
{context}
</context>

<question>
{question}
</question>

Analyze the philosophical concept using ONLY the provided context. Follow these steps:
1. Identify key elements from the context
2. Explain relationships between concepts
3. Highlight any mentioned philosophical implications

If context is insufficient, state: "This requires deeper analysis beyond current knowledge." 

Answer:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)

# 5. Optimized retrieval configuration
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 6,
            "score_threshold": 0.55,
            "include_metadata": True
        }
    ),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
        "document_prompt": PromptTemplate(
            input_variables=["page_content"],
            template="{page_content}"
        )
    },
    verbose=True
)

# 6. Query processing function
def process_query(query):
    if not query.strip():
        return "Please enter a valid question", ""
    
    try:
        result = qa_chain({"query": query})
        response = result['result'].split("Answer:")[-1].strip()
        
        sources_info = "\n\n**Sources:**\n"
        for i, doc in enumerate(result['source_documents'][:3], 1):
            excerpt = doc.page_content[:200].replace("\n", " ") + "..."
            score = doc.metadata.get('score', 0)
            sources_info += f"{i}. {excerpt} (Relevance: {score:.2f})\n\n"
            
        return response, sources_info
    
    except Exception as e:
        return f"Error processing request: {str(e)}", ""

# 7. Gradio interface
def create_gradio_interface():
    with gr.Blocks(title="Philosophical Concepts RAG v2") as demo:
        gr.Markdown("# Enhanced Philosophical RAG System")
        gr.Markdown("Now with improved context understanding and source verification")
        
        with gr.Row():
            with gr.Column(scale=4):
                query_input = gr.Textbox(
                    label="Your Question",
                    placeholder="Ask about philosophical concepts...",
                    lines=3
                )
                submit_btn = gr.Button("Analyze", variant="primary")
            
        with gr.Row():
            with gr.Column(scale=3):
                response_output = gr.Markdown(label="Analysis")
            with gr.Column(scale=2):
                sources_output = gr.Markdown(label="Supporting Context")
                
        submit_btn.click(
            fn=process_query,
            inputs=[query_input],
            outputs=[response_output, sources_output]
        )
        
        gr.Markdown("## Example Queries")
        examples = gr.Examples(
            examples=[
                ["Explain the relationship between consciousness and reality using the provided context"],
                ["How does the concept of 'being' differ across philosophical traditions?"],
                ["Analyze the arguments for and against epistemological dualism"]
            ],
            inputs=query_input
        )
        
    return demo

# 8. Launch the interface
if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True)

FAISS index created with 150 entries


model.safetensors:   4%|4         | 231M/5.31G [00:00<?, ?B/s]